In [2]:
from Dataset_v1 import Dataset_v1
from Model_v1 import Model_v1
from Analysis import FeatureAnalysis 
from ModelEval import ModelEval
import pandas as pd
import numpy as np

In [13]:
train_data = pd.read_csv("data/train_data_new.csv")
test_data = pd.read_csv("data/test_data_new.csv")
data_base = pd.concat([train_data,test_data])

In [2]:
dataset = Dataset_v1("data/train_data_new.csv","data/test_data_new.csv",False)
dataset.build_features_into_dataset()
analysis = FeatureAnalysis(dataset)
data=dataset.get_data()

Non numeric features:
 ['BroadcastName', 'TeamName', 'Country', 'Location']


In [32]:
#data.loc[(data["Race_Time_Encoding"]==1)&(data["target"]==1),["Location","Round_Number","Year","BroadcastName","Qual_Q3_Time_Normal","Qual_Position","Race_Time_Encoding","Race_Position"]]

In [9]:
def check_standardized_time_ordering(df):
    def is_time_ascending(group):
        # Sort by Race_Position
        group_sorted = group.sort_values("Qual_Position", ascending=True)
        group_sorted.replace(-1,1000,inplace=True)
        # Check if Standardized_Time is non-decreasing
        valid_times = group_sorted["Qual_Q3_Time"].dropna()
        return valid_times.is_monotonic_increasing

    # Apply the check to each (Year, Round_Number) group
    result = (
        df.groupby(["Year", "Round_Number"])
        .apply(is_time_ascending)
        .reset_index(name="is_time_ordered")
    )

    return result

In [11]:
res = check_standardized_time_ordering(data_base)
res.loc[res["is_time_ordered"]==False]

,Year,Round_Number,is_time_ordered
58,2024,15,False
74,2025,7,False


In [16]:
row_condition = (data_base["Year"]==2025)&(data_base["Round_Number"]==7) 
data_base.loc[row_condition,["Location","TeamName","BroadcastName","Qual_Q3_Time","Qual_Position","Standardized_Time","Race_Position"]].head(20).sort_values(by="Race_Position")

,Location,TeamName,BroadcastName,Qual_Q3_Time,Qual_Position,Standardized_Time,Race_Position
120,Imola,Red Bull Racing,M VERSTAPPEN,74.704,2.0,0.000,1.0
131,Imola,McLaren,L NORRIS,-1.000,4.0,6.109,2.0
138,Imola,McLaren,O PIASTRI,74.670,1.0,12.956,3.0
133,Imola,Ferrari,L HAMILTON,-1.000,12.0,14.356,4.0
127,Imola,Williams,A ALBON,75.473,7.0,17.945,5.0
124,Imola,Ferrari,C LECLERC,-1.000,11.0,20.774,6.0
137,Imola,Mercedes,G RUSSELL,74.807,3.0,22.034,7.0
135,Imola,Williams,C SAINZ,75.432,6.0,22.898,8.0
136,Imola,Racing Bulls,I HADJAR,75.746,9.0,23.586,9.0
126,Imola,Red Bull Racing,Y TSUNODA,-1.000,20.0,26.446,10.0


In [27]:

def yuh(year,round):
    row_condition = (data["Year"]==year)&(data["Round_Number"]==round)
    info = data.loc[row_condition,["Location","Year","Round_Number","BroadcastName","n_past","Qual_Position","driver_encoding","Race_Position"]].sort_values(by="Qual_Position")
    print(len(info))
    return info 

In [28]:
#data[(data['BroadcastName'] == 'S PEREZ') & (data['Year'] == 2024)][['Race_Date_Code', 'Race_Position', 'n_past', 'driver_encoding']]


In [36]:
yuh(2022,15) 

20


,Location,Year,Round_Number,BroadcastName,n_past,Qual_Position,driver_encoding,Race_Position
243,Zandvoort,2022,15,M VERSTAPPEN,14,1.0,6.873841,1.0
254,Zandvoort,2022,15,C LECLERC,14,2.0,9.350008,3.0
238,Zandvoort,2022,15,C SAINZ,14,3.0,8.505778,8.0
252,Zandvoort,2022,15,L HAMILTON,14,4.0,9.502775,4.0
257,Zandvoort,2022,15,S PEREZ,14,5.0,8.529517,5.0
239,Zandvoort,2022,15,G RUSSELL,14,6.0,8.066105,2.0
255,Zandvoort,2022,15,L NORRIS,14,7.0,9.874931,7.0
251,Zandvoort,2022,15,M SCHUMACHER,14,8.0,11.973969,13.0
241,Zandvoort,2022,15,Y TSUNODA,14,9.0,12.723111,20.0
242,Zandvoort,2022,15,L STROLL,14,10.0,10.820291,10.0


In [3]:
### MODEL EVAL EXPERIMENTING ###

dataset = Dataset_v1("data/train_data_new.csv","data/test_data_new.csv",False)
dataset.build_features_into_dataset()

features_for_training = [
    "n_past_podiums_last_5",
    "Qual_Position",
    #"ewa_driver_results",
    "driver_encoding",
    #"Race_Time_Encoding",
    #"Qual_Q3_Time_Normal",
    #"pos_gained_encoding_simple",
    #"n_past_podiums",
    #"pos_gained_encoding",
    #"TopTeam_Red Bull Racing",
    #"TopTeam_Ferrari",
    #"TopTeam_McLaren",
    #"Sprint_Race_Position",
    #"Sprint_Qual_Position",
    #"TopTeam_Mercedes"
]

dataset.set_features_for_training(features_for_training)

model = Model_v1(dataset,"RF_trn",False)
model.train()
print(model.get_feature_importance())


Features for training set:
 ['n_past_podiums_last_5', 'Qual_Position', 'driver_encoding']

Dataset Shapes
(1221, 3) (1221,)
(236, 3) (236,)
(153, 3) (153,)


Finding best threshold...
Precision: 0.5813953488372093
Recall: 0.6944444444444444
F1 score: 0.6410256410256411
Best PR threshold: 0.8212164057215823


FPR: 0.245
TPR: 0.9444444444444444
Best ROC threshold: 0.39437358835770886


Best threshold: 0.8212164057215823
Threshold set to: 0.8212164057215823
                 feature  importance
1          Qual_Position    0.424129
2        driver_encoding    0.291984
0  n_past_podiums_last_5    0.283887


In [4]:
model_eval = ModelEval(model,dataset)
model_eval_df = model_eval.get_df()

In [5]:
model_eval.get_preds_for_race(2025,7)

,n_past_podiums_last_5,Qual_Position,driver_encoding,Round_Number,Year,Location,Driver,Actual,Predicted,Probability
1426,5.0,1.0,4.546112,7,2025,Imola,O PIASTRI,1,1,0.971795
1431,2.0,2.0,4.940751,7,2025,Imola,M VERSTAPPEN,1,1,0.949795
1425,3.0,3.0,5.087376,7,2025,Imola,G RUSSELL,0,1,0.907562
1434,4.0,4.0,4.447482,7,2025,Imola,L NORRIS,1,1,0.905765
1430,1.0,11.0,6.793254,7,2025,Imola,C LECLERC,0,0,0.326851
1424,0.0,12.0,8.545097,7,2025,Imola,L HAMILTON,0,0,0.307819
1427,0.0,5.0,12.799182,7,2025,Imola,F ALONSO,0,0,0.258605
1420,0.0,13.0,9.601316,7,2025,Imola,K ANTONELLI,0,0,0.206606
1433,0.0,7.0,8.988225,7,2025,Imola,A ALBON,0,0,0.080107
1419,0.0,6.0,10.761427,7,2025,Imola,C SAINZ,0,0,0.063574


In [6]:
model_eval.get_driver_with_most_wrong_predictions()

Driver with the most wrong predictions: G RUSSELL 0.7027027027027027


,n_past_podiums_last_5,Qual_Position,driver_encoding,Round_Number,Year,Location,Driver,Actual,Predicted,Probability,correct
6,0.0,6.0,9.931745,3,2022,Melbourne,G RUSSELL,1,0,0.332577,False
26,1.0,11.0,9.604824,4,2022,Imola,G RUSSELL,0,0,0.090275,True
53,1.0,12.0,9.374596,5,2022,Miami,G RUSSELL,0,0,0.125321,True
58,1.0,4.0,9.234396,6,2022,Barcelona,G RUSSELL,1,0,0.764924,False
87,2.0,6.0,8.942308,7,2022,Monaco,G RUSSELL,0,0,0.493084,True
...,...,...,...,...,...,...,...,...,...,...,...
1366,2.0,2.0,5.461310,4,2025,Sakhir,G RUSSELL,1,1,0.960989,True
1379,3.0,3.0,4.964327,5,2025,Jeddah,G RUSSELL,0,1,0.906751,False
1400,3.0,5.0,5.316177,6,2025,Miami,G RUSSELL,1,0,0.805819,False
1425,3.0,3.0,5.087376,7,2025,Imola,G RUSSELL,0,1,0.907562,False


In [11]:
gr = model_eval.get_driver_with_most_wrong_predictions()
gr.loc[gr["Year"]==2025,:]

Driver with the most wrong predictions: G RUSSELL 0.7027027027027027


,n_past_podiums_last_5,Qual_Position,driver_encoding,Round_Number,Year,Location,Driver,Actual,Predicted,Probability,correct
1317,1.0,4.0,5.714026,1,2025,Melbourne,G RUSSELL,1,1,0.881845,True
1323,2.0,2.0,5.392158,2,2025,Shanghai,G RUSSELL,1,1,0.960989,True
1348,3.0,5.0,5.160811,3,2025,Suzuka,G RUSSELL,0,0,0.796326,True
1366,2.0,2.0,5.461310,4,2025,Sakhir,G RUSSELL,1,1,0.960989,True
1379,3.0,3.0,4.964327,5,2025,Jeddah,G RUSSELL,0,1,0.906751,False
1400,3.0,5.0,5.316177,6,2025,Miami,G RUSSELL,1,0,0.805819,False
1425,3.0,3.0,5.087376,7,2025,Imola,G RUSSELL,0,1,0.907562,False
1441,2.0,14.0,5.869363,8,2025,Monaco,G RUSSELL,0,0,0.414228,True


In [8]:
model_eval.get_drivers_with_most_predictions()

,Driver,count
6,M VERSTAPPEN,58
0,C LECLERC,31
5,L NORRIS,30
3,G RUSSELL,18
8,S PEREZ,16
1,C SAINZ,15
7,O PIASTRI,14
4,L HAMILTON,11
2,F ALONSO,3


In [10]:
#analysis.plot_prob_dist("Race_Time_Encoding")

In [ ]:
analysis.plot_prob_dist("driver_encoding")

In [ ]:
#analysis.plot_prob_dist("lag_SpeedST")

In [ ]:
#analysis.plot_prob_dist("ewa_driver_results")

In [ ]:
def create_pos_gained_encoding(driver, qual_position, race_position):
    """
    # Convert inputs to lists if they're pandas Series
    if isinstance(driver, pd.Series):
        driver = driver.tolist()
    if isinstance(qual_position, pd.Series):
        qual_position = qual_position.tolist()
    if isinstance(race_position, pd.Series):
        race_position = race_position.tolist()
    """
    
    # Handle -1 in race_position
    race_position = [21 if x == -1 else x for x in race_position]
    
    # Calculate position gained
    pos_gained = [q - r for q, r in zip(qual_position, race_position)]

    def calc(x): return (1/np.log(x+1))*((21-x)/20)
    
    # Calculate finishing weights
    finishing_weight = [calc(x) for x in race_position]
    
    final = [] 
    podium_bonus = 1

    for i in range(len(pos_gained)):
        if pos_gained[i] == 0:
            adjusted_pos_gained = finishing_weight[i]
        elif pos_gained[i] > 0:
            adjusted_pos_gained = pos_gained[i]*finishing_weight[i]
        else:
            if finishing_weight[i] <= 0.0: 
                adjusted_pos_gained = pos_gained[i]*(-np.log(finishing_weight[i]+0.000001))*0.05
            else:
                adjusted_pos_gained = pos_gained[i]*(-np.log(finishing_weight[i]))*0.05
            
        if race_position[i] <= 3:
            final.append(adjusted_pos_gained + podium_bonus)
        else:
            final.append(adjusted_pos_gained)

    df = pd.DataFrame({
        "driver": driver,
        "qual_position": qual_position,
        "race_pos": race_position,
        "pos_gained": pos_gained,
        "finishing_weight": finishing_weight,
        "final": final
    }).sort_values(by="final", ascending=False)
    
    return df

In [ ]:
race_results = data.loc[:, ['BroadcastName','Qual_Position','Race_Position']].iloc[20:40]
create_pos_gained_encoding(race_results["BroadcastName"],race_results["Qual_Position"],race_results["Race_Position"])

In [ ]:
race_results = data.loc[:, ['BroadcastName','Qual_Position','Race_Position']].iloc[0:20]
create_pos_gained_encoding(race_results["BroadcastName"],race_results["Qual_Position"],race_results["Race_Position"])